In [1]:
%cd ../..

/home/andrea/Projects/piacere-model-checker


In [21]:
# XMI_PATH = "./tests/doml/CaseStudies/posidonia.domlx"
# XMI_PATH = "./tests/doml/CaseStudies/posidonia_IOP.domlx" # Subnets, no Gateway
XMI_PATH = "../piacere-workspace/Playground/CaseStudies/nio3_test_exec_env.domlx" # No subnets, Gateway
# XMI_PATH = "../piacere-workspace/Playground/CaseStudies/posidonia.domlx"

with open(XMI_PATH, 'rb') as f:
    doml_xmi = f.read()

In [22]:
from mc_openapi.doml_mc.main import init_model

dmc = init_model(doml_xmi, doml_ver=None)

In [23]:
# from mc_openapi.doml_mc.network_address_validator import validate_network_address

# validate_network_address(dmc)

In [29]:
from pprint import pprint
from mc_openapi.doml_mc.intermediate_model import DOMLElement
from mc_openapi.doml_mc.mc import ModelChecker

from ipaddress import IPv4Address, IPv4Network

ASSOC_SUBNETS = 'infrastructure_Network::subnets'
ASSOC_GATEWAYS = 'infrastructure_InternetGateway::address'
ASSOC_IFACE_NET = 'infrastructure_NetworkInterface::belongsTo'

ATTR_NET_CIDR = 'infrastructure_Network::cidr'
ATTR_GATEWAY_ADDRESS = 'infrastructure_InternetGateway::address'
ATTR_IFACE_ADDRESS = 'infrastructure_NetworkInterface::endPoint'

def get_attr(elem: DOMLElement, attr_id: str):
    if elem := elem.attributes.get(attr_id):
        return elem[0]
    return None
    
def get_assocs(elem: DOMLElement, assoc_id: str):
    return elem.associations.get(assoc_id, [])

def validate_network_address(dmc: ModelChecker):
    im = dmc.intermediate_model

    networks = [e for e in im.values() if e.class_ == 'infrastructure_Network']
    gateways = [e for e in im.values() if e.class_ == 'infrastructure_InternetGateway']
    ifaces = [e for e in im.values() if e.class_ == 'infrastructure_NetworkInterface']
    subnets = [e for e in im.values() if e.class_ == 'infrastructure_Subnet']

# Network/24
#   Subnet/28
#   Subnet/30
#   Address
#   Address

    def visit_subnet(net: DOMLElement, acc: list):
        """Recursively navigate subnets to populate the `acc` list with all the subnet in a network."""
        for subnet in get_assocs(net, ASSOC_SUBNETS):
            subnet = im[subnet]
            acc.append((subnet, get_attr(subnet, ATTR_NET_CIDR)))
            visit_subnet(subnet, acc)

    for network in networks:
        # Tuple(elem, cidr)
        subnets: list[tuple] = []
        # Tuple(elem, address)
        addresses: list[tuple] = []

        visit_subnet(network, subnets)

        for gateway in get_assocs(network, ASSOC_GATEWAYS):
            gateway = im[gateway]
            if gateway_address := get_attr(gateway, ATTR_GATEWAY_ADDRESS):
                addresses.append((gateway, IPv4Address(gateway_address)))

        for iface in ifaces:
            if owner := get_assocs(iface, ASSOC_IFACE_NET):
                owner_id = im[list(owner)[0]].id_
                if (owner_id in [s for (s, _) in subnets] 
                or owner_id == network.id_):
                    if iface_address := get_attr(iface, ATTR_IFACE_ADDRESS):
                        addresses.append((iface, IPv4Address(iface_address)))

        pprint(network.user_friendly_name)
        pprint(subnets)
        pprint(addresses)


validate_network_address(dmc)

'net1'
[]
[(DOMLElement(id_='elem_140262387532240',
              class_='infrastructure_NetworkInterface',
              attributes={'commons_DOMLElement::name': ['i1'],
                          'infrastructure_NetworkInterface::endPoint': [173216344]},
              associations={'infrastructure_NetworkInterface::belongsTo': {'elem_140262378799888'}},
              user_friendly_name='i1'),
  IPv4Address('10.83.18.88')),
 (DOMLElement(id_='elem_140262380465168',
              class_='infrastructure_NetworkInterface',
              attributes={'commons_DOMLElement::name': ['i1'],
                          'infrastructure_NetworkInterface::endPoint': [173216348]},
              associations={'infrastructure_NetworkInterface::belongsTo': {'elem_140262378799888'}},
              user_friendly_name='i1'),
  IPv4Address('10.83.18.92'))]
